In [1]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, matthews_corrcoef, recall_score, precision_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import roc_curve
import torch.nn.functional as F

In [2]:
loaded_datasets_info = torch.load('/root/autodl-tmp/data/saved_datasets.pth', weights_only=False)
train_dataset = loaded_datasets_info['train_dataset']
test_dataset = loaded_datasets_info['test_dataset']

In [3]:
batch_size = 100
loaded_train_dataset = DataLoader(train_dataset, batch_size = batch_size, shuffle = False)
loaded_test_dataset = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [4]:
class Rnn(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_layer, n_classes):
        super(Rnn, self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layer, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, n_classes)

    def forward(self, x):
        out, (h_n, c_n) = self.lstm(x)
        # 此时可以从out中获得最终输出的状态h
        # x = out[:, -1, :]
        x = h_n[-1, :, :]
        x = self.classifier(x)
        return x

In [5]:
device = "cuda"
model = Rnn(28, 10, 2, 10).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
num_epochs = 200

In [6]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_indx, (inputs, labels) in enumerate(loaded_train_dataset):
        inputs = inputs.squeeze(1).to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    # Print average loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / (len(loaded_train_dataset) / batch_size)}")  

Epoch 1/200, Loss: 126.19795826077461
Epoch 2/200, Loss: 66.11290494849284
Epoch 3/200, Loss: 52.597390815615654
Epoch 4/200, Loss: 47.459223276625075
Epoch 5/200, Loss: 38.547007297476135
Epoch 6/200, Loss: 37.162990103165306
Epoch 7/200, Loss: 36.307579005757965
Epoch 8/200, Loss: 36.78123012805978
Epoch 9/200, Loss: 32.717815804605685
Epoch 10/200, Loss: 32.46670545016726
Epoch 11/200, Loss: 38.18728304281831
Epoch 12/200, Loss: 33.91482072571913
Epoch 13/200, Loss: 33.838294779260956
Epoch 14/200, Loss: 29.267536561315257
Epoch 15/200, Loss: 27.233152664266527
Epoch 16/200, Loss: 26.681678202624123
Epoch 17/200, Loss: 28.460340965539217
Epoch 18/200, Loss: 26.445654776568215
Epoch 19/200, Loss: 25.904589142029483
Epoch 20/200, Loss: 25.09128775075078
Epoch 21/200, Loss: 24.711430778416496
Epoch 22/200, Loss: 24.713333634970088
Epoch 23/200, Loss: 23.924099241072934
Epoch 24/200, Loss: 27.321048800523084
Epoch 25/200, Loss: 29.47803341348966
Epoch 26/200, Loss: 30.50377452125152
Epo

In [7]:
torch.save(model.state_dict(), './model_params/LSTM.pth')

In [8]:
model.load_state_dict(torch.load('./model_params/LSTM.pth'))

/tmp/ipykernel_1289/3218175715.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./model_params/LSTM.pth'))


<All keys matched successfully>

### Metrics function definition

In [31]:
# get roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC
def calculate_multiclass_metrics(true_labels, predicted_labels, predicted_probabilities, num_classes):
    accuracy = accuracy_score(true_labels, predicted_labels)
    mcc = matthews_corrcoef(true_labels, predicted_labels)
    
    sensitivity_per_class = []
    specificity_per_class = []
    auc_per_class = []
    f1_per_class = []

    for i in range(num_classes):
        true_binary = (np.array(true_labels) == i).astype(int)
        pred_binary = (np.array(predicted_labels) == i).astype(int)

        cm = confusion_matrix(true_binary, pred_binary, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()

        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity_per_class.append(sensitivity)
        specificity_per_class.append(specificity)

        
        auc = roc_auc_score(true_binary, predicted_probabilities[:, i]) if len(np.unique(true_binary)) > 1 else 0
        auc_per_class.append(auc)

        f1 = f1_score(true_binary, pred_binary) if len(np.unique(true_binary)) > 1 else 0
        f1_per_class.append(f1)

    avg_sensitivity = np.mean(sensitivity_per_class)
    avg_specificity = np.mean(specificity_per_class)
    avg_auc = np.mean(auc_per_class) if auc_per_class else 0
    avg_f1 = np.mean(f1_per_class)

    print(f"Average AUC: {avg_auc:.4f}")
    print(f"Average Sensitivity: {avg_sensitivity:.4f}")
    print(f"Average Specificity: {avg_specificity:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Average F1-score: {avg_f1:.4f}")
    print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

### Training data metrics

In [32]:
predicted_probabilities = []
true_labels = []
with torch.set_grad_enabled(False):
    for batch_indx, (inputs, labels) in enumerate(loaded_train_dataset):
        inputs = inputs.squeeze(1).to(device)
        labels = labels.to(device)      
        outputs = model(inputs)
        predicted_probabilities.extend(outputs.tolist())
        true_labels.extend(labels.tolist())

In [33]:
true_labels_ = np.argmax(true_labels, axis=-1)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)
preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)

train_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=10)

Average AUC: 0.9978
Average Sensitivity: 0.9470
Average Specificity: 0.9942
Accuracy: 0.9473
Average F1-score: 0.9468
Matthews Correlation Coefficient (MCC): 0.9415


In [34]:
np.save('/root/autodl-tmp/ROC/LSTM/y_train_pred.npy', preds)
np.save('/root/autodl-tmp/ROC/LSTM/y_train.npy', true_labels)

### Testing data metrics

In [35]:
predicted_probabilities = []  
true_labels = []  
with torch.set_grad_enabled(False): 
    for batch_indx, (inputs, labels) in enumerate(loaded_test_dataset):
        inputs = inputs.squeeze(1).to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
        predicted_probabilities.extend(outputs.tolist())
        true_labels.extend(labels.tolist())

In [36]:
true_labels_ = np.argmax(true_labels, axis=-1)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)
preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)

test_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=10)

Average AUC: 0.9980
Average Sensitivity: 0.9447
Average Specificity: 0.9939
Accuracy: 0.9449
Average F1-score: 0.9444
Matthews Correlation Coefficient (MCC): 0.9389


In [37]:
np.save('/root/autodl-tmp/ROC/LSTM/y_test_pred.npy', preds)
np.save('/root/autodl-tmp/ROC/LSTM/y_test.npy', true_labels)